### Steering PID Controller - Analysis

In [6]:
import json
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearAxis, Range1d, Title

# Enable Bokeh output in the notebook
output_notebook()

Loading BokehJS ...

In [29]:
def analyze_file(path):
    # Read the log file
    with open(path, 'r') as f:
        lines = f.readlines()

    # Parse JSON objects and extract PID data points
    data = []
    config_text = ""
    basetime = None
    for line in lines:
        try:
            obj = json.loads(line)
            if obj.get('type') == 'pid' and obj.get('response') < 750 and obj.get('response') > -750:
                if basetime is None:
                    basetime = obj.get('timestamp')
                data.append({
                    'timestamp': obj.get('timestamp') - basetime,
                    'error': obj.get('error'),
                    'response': obj.get('response'),
                    'P': obj.get('P'),
                    'I': obj.get('I'),
                    'D': obj.get('D')
                })
            elif obj.get('type') == 'configuration':
                config_text = f"Kp = {obj.get('Kp')}, Ki = {obj.get('Ki')}, Kd = {obj.get('Kd')}"
        except json.JSONDecodeError:
            continue
    
    # Convert to DataFrame for easier plotting
    df = pd.DataFrame(data)
    
    # Skip plotting if no filtered data
    if df.empty:
        print("No data points with response < 750 found")
    else:
        # Find range for primary y-axis (response, P, I, D)
        primary_min = min(
            df['response'].min(),
            df['P'].min(),
            df['I'].min(),
            df['D'].min()
        )
        primary_max = max(
            df['response'].max(),
            df['P'].max(),
            df['I'].max(),
            df['D'].max()
        )
    
        # Find range for secondary y-axis (error)
        error_min = df['error'].min()
        error_max = df['error'].max()
    
        # Calculate scaling factors to align the zero points
        # First determine the relative positions of zero in both ranges
        if primary_min >= 0:
            primary_zero_pos = 0
        elif primary_max <= 0:
            primary_zero_pos = 1
        else:
            primary_zero_pos = abs(primary_min) / (primary_max - primary_min)
    
        if error_min >= 0:
            error_zero_pos = 0
        elif error_max <= 0:
            error_zero_pos = 1
        else:
            error_zero_pos = abs(error_min) / (error_max - error_min)
    
        # Adjust ranges to align zero points
        if primary_zero_pos < error_zero_pos:
            # Expand primary range downward
            new_primary_min = -primary_max * error_zero_pos / (1 - error_zero_pos)
            primary_range = (new_primary_min, primary_max)
            error_range = (error_min, error_max)
        elif primary_zero_pos > error_zero_pos:
            # Expand error range downward
            new_error_min = -error_max * primary_zero_pos / (1 - primary_zero_pos)
            primary_range = (primary_min, primary_max)
            error_range = (new_error_min, error_max)
        else:
            # Zero points already aligned
            primary_range = (primary_min, primary_max)
            error_range = (error_min, error_max)
    
        # Add padding to both ranges
        def add_padding(range_tuple, padding_percent=0.1):
            min_val, max_val = range_tuple
            padding = (max_val - min_val) * padding_percent
            return (min_val - padding, max_val + padding)
    
        primary_range_padded = add_padding(primary_range)
        error_range_padded = add_padding(error_range)
    
        # Create a Bokeh figure with primary y-axis
        p = figure(
            title="Steering PID Analysis",
            x_axis_label="Timestamp",
            y_axis_label="Values (Response, P, I, D)",
            y_range=primary_range_padded,
            width=800,
            height=500
        )
        
        # Add the configuration as a subtitle if found
        if config_text:
            p.add_layout(Title(text=config_text, text_font_style="italic"), "above")
    
        # Add secondary y-axis for error
        p.extra_y_ranges = {"error_axis": Range1d(*error_range_padded)}
        p.add_layout(LinearAxis(y_range_name="error_axis", axis_label="Error"), 'right')
    
        # Add a horizontal line at y=0 for reference
        p.line(x=[df['timestamp'].min(), df['timestamp'].max()], y=[0, 0], 
               line_width=1, color='black', alpha=0.5, line_dash='dashed')
    
        # Plot error line using the secondary y-axis
        p.line(df['timestamp'], df['error'], line_width=2, color='red', 
               legend_label='Error', y_range_name="error_axis")
    
        # Plot other lines using the primary y-axis
        p.line(df['timestamp'], df['response'], line_width=2, color='green', legend_label='Response')
        p.line(df['timestamp'], df['P'], line_width=2, color='orange', legend_label='P')
        p.line(df['timestamp'], df['I'], line_width=2, color='blue', legend_label='I')
        p.line(df['timestamp'], df['D'], line_width=2, color='purple', legend_label='D')
    
        # Configure the legend
        p.legend.location = "top_right"
        p.legend.click_policy = "hide"  # Allow toggling lines on/off by clicking legend
    
        # Show the plot
        show(p)

In [35]:
analyze_file('logs/pid.7.log')

In [47]:
analyze_file('logs/pid.15.log')

In [48]:
analyze_file('logs/pid.16.log')

In [49]:
analyze_file('logs/pid.17.log')

In [50]:
analyze_file('logs/pid.18.log')

In [51]:
analyze_file('logs/pid.19.log')

In [52]:
analyze_file('logs/pid.20.log')

In [54]:
analyze_file('logs/pid.21.log')

In [55]:
analyze_file('logs/pid.22.log')